In [1]:
import json
import os
import pandas as pd
import numpy as np
from pathlib import Path
import collections
from sklearn.model_selection import train_test_split
from sklearn import metrics

import sys
sys.path.append("../")
from datatools.analyzer import *
from utterance.error_tools import *

from datatools.maneger import DataManager
from datatools.preproc import Preprocessor

/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.26.5) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
def is_declinable(token):
    pos = token.tag_.split("-")
    try:
        if pos[0] == "動詞" or pos[0] == "形容詞" or pos[2] == "サ変可能":
            # print(token.lemma_)
            return True
        else:
            False
    except:
        return False

In [3]:
def depend_extract(t): 
    doc = nlp(t)
    for sent in doc.sents:
        for token in sent:
            # if is_declinable(token):
            if token.pos_ == "VERB":
                    # print()
                    # register_pred_count(pred_cound_dict, token.lemma_)
                for c in token.children:
                        # print(c)
                    for c2 in c.children:
                        if c2.dep_ == "case":
        
                            print( c.lemma_, c2, token.lemma_)
                            # register_pred_case_lite(pred_case_count_dict, token.lemma_, c2.text, c)
                            # print()

In [4]:
t = "インターネットは排出量が多い"
t = "マドリッドの原宿、表参道エリアを中心に人気専門店が続々登場するなど、今年一躍話題になったポップコーンですが、日本穀物検定協会が金曜ロードSHOW!とコラボレーションしたきょうは会社休みます。を12月に発売するそうです。"
t = "固有抽出表現は、文から日付や地名等の固有表現を抽出する処理です。"
t = "天気が外に出ますよね"

In [5]:
def extract_continue_noun(c):
    noun = c.text
    C = c
    for left in reversed(list( c.lefts) ):
        if left.pos_ in ["NOUN", "PROPN", "NUM"]:
            # noun = left.text + noun
            noun = extract_continue_noun(left) + noun
        else:
            break
        # print("\t\t", left, end=", ")
    # print()
    return noun

In [6]:
import ginza

doc = nlp(t)
for token in doc:
    if token.pos_ in ["VERB", "ADJ"]:
        print(token.lemma_)
        for c in token.children:
            if "名詞" in c.tag_:
                # print("\t", c.lemma_, c.dep_)
                noun = extract_continue_noun(c)
                # print("\t", extract_continue_noun(c), c.dep_)
                # print("\t", extract_continue_noun(c))
                for c2 in c.children:
                    if c2.dep_ == "case":
                        print( c, c2, token.lemma_)
        print()

出る
天気 が 出る
外 に 出る



In [7]:
from spacy import displacy
displacy.render(nlp("旅行に参加しちゃった"), style='dep', jupyter=True, options={'compact':True, 'distance': 90})

In [8]:
path = "../hand_labeled/"
datalist = ['DCM', 'DIT', 'IRS']
convs = read_conv(path, datalist)

In [9]:
error = "Semantic error"
sys_utt = []
y = []
for conv in convs:
    for ut in conv:
        if ut.is_system() and ut.is_exist_error():
            sys_utt.append(ut.utt)
            if ut.is_error_included(error):
                y.append(1)
            else:
                y.append(0)

In [10]:
def RDF_test(text):
    text = clean_text(text)
    doc = nlp(text)
    phrases = ""
    for i, token in enumerate( doc ):
        # phrases.append("")
        if token.pos_ in ["VERB", "ADJ"]:
            # print(token.lemma_)
            phrase = ""
            for c in token.children:
                if "名詞" in c.tag_:
                    # print("\t", c.lemma_, c.dep_)
                    noun = extract_continue_noun(c)
                    # print("\t", extract_continue_noun(c), c.dep_)
                    # print("\t", extract_continue_noun(c))
                    for c2 in c.children:
                        if c2.dep_ == "case":
                            # print("\t", c, c2, token.lemma_)
                            phrase = phrase + noun+ c2.orth_

            # トリプルがない
            if phrase=="":
                continue
            phrase += token.lemma_
            # 形容詞 
            if (token.pos_=="ADJ" or "名詞" in token.tag_) and i<len(doc)-1:
                phrase += doc[i+1].lemma_
            
            # 連結
            if phrases=="":
                phrases = phrase
            else:
                phrases = phrases+"。"+phrase

            # print(phrase)
    return phrases
            

In [11]:
RDF_test("旅行を検討しちゃった")

'旅行を検討する'

In [12]:
rdf_list = []
for utt in sys_utt:
    print(utt)
    rdf_list.append(RDF_test(utt))
    # print()

海に行かないんですね？
海は大丈夫ですね
海は危険です
海はいけないです
わあ！いいですね！
ご存知ですいます
ご存知になられます
スイカは実がなってたことあります
あなたの好きな映画について教えてください
映画にはまるのです
映画が楽しみです
映画が楽しみですね
映画館はチケットを買うのです
食事で摂るのです
外でとります
外食は二人で行きますねぇ
食事のパートナーはいてるんですね。外食で焼肉に行きますねぇ
あなた、外食のときも私を連れて行ってよ。
夕食は明日の活力を生むから、しっかり食べてね。
野菜はカロリーが低いですよねえ
カロリーは欲しいですね
ラーメンが食べたいです？
ラーメンは美味しいですね
麺が美味しいですね
パスタが好きですね
パスタはクリーム系が好きですね
海は楽しいですね
海はサイコーですね
海はキレーですね
魚はおいんですか？？海は素晴らしいですね
海は素晴らしいですね
台風が凄いです
熱中症はいいですね
熱中症は大丈夫ですね
手洗いうがいで予防が大切ですよねえ
予防は手洗いが基本なんですよねー
予防は難しいですね
海に行かないんですね？
海に行ってるんですね。海は一人で行ってます？
家族で買い物に行きます？
家族でどこかに出かけます？
家族で食べに行きますか？
家族で初詣に行ってるんですね。家族で旅行に行ってるのですか？
家族は夕食を食べてるのですか？
ケースバイケースで使えるのです
ケースバイケースで恋愛スキルを上げたいのです
必要とケースバイケースになってるんですね。ケースバイケースを見ます
ケースバイケースで判断するんですね
そういえば、よく旅行に行くと言っていましたが、ケースバイケースで臨機応変に対応するのでしょうか？
お名前はco.jpに添えます
旅行に参加しちゃった
旅行を検討しちゃった
北海道をはじめるかもしれない
大分別府まで九州横断特急に乗るんでしょうか？
奇遇に驚きますか？
しりとりは一緒に盛り上がりましょうか？
プレゼントは、いつでも嬉しいかな。
紅玉を求めるかもしれない
そういえば、この間、旅行したと伺いましたが、奇遇に驚きます
旅行が出来るのです
最近興味のあることは何ですか？
朝から天気が変わりますよねー
広すぎて、考えるのもイヤになるよ。
もちろんです
付近は紅葉が進むようです
恋の季節って、やっぱり夏なのかなあ

In [13]:
import logging

import torch
from transformers import BertForMaskedLM, BertJapaneseTokenizer

logger = logging.getLogger(__name__)


class BertProofreader:
    def __init__(self, pretrained_model: str, cache_dir: str = None):

        # Load pre-trained model tokenizer (vocabulary)
        self.tokenizer = BertJapaneseTokenizer.from_pretrained(pretrained_model, cache_dir=cache_dir)

        # Load pre-trained model (weights)
        self.model = BertForMaskedLM.from_pretrained(pretrained_model, cache_dir=cache_dir)
        self.model.to('cuda')

        self.model.eval()

    def mask_prediction(self, sentence: str) -> torch.Tensor:
        # 特殊Tokenの追加
        sentence = f'[CLS]{sentence}[SEP]'

        tokenized_text = self.tokenizer.tokenize(sentence)

        indexed_tokens = self.tokenizer.convert_tokens_to_ids(tokenized_text)
        tokens_tensor = torch.tensor([indexed_tokens], device='cuda')

        # [MASK]に対応するindexを取得
        mask_index = self.tokenizer.convert_tokens_to_ids(['[MASK]'])[0]

        # 1単語ずつ[MASK]に置き換えたTensorを作る
        repeat_num = tokens_tensor.shape[1] - 2
        tokens_tensor = tokens_tensor.repeat(repeat_num, 1)
        for i in range(repeat_num):
            tokens_tensor[i, i + 1] = mask_index

        # Predict all tokens
        with torch.no_grad():
            outputs = self.model(tokens_tensor, token_type_ids=None)
            predictions = outputs[0]

        return tokenized_text, predictions

    def check_topk(self, sentence: str, topk: int = 10):
        """
        [MASK]に対して予測された単語のTop Kに元の単語が含まれていればTrueと判定
        """

        tokens, predictions = self.mask_prediction(sentence)

        pred_sort = torch.argsort(predictions, dim=2, descending=True)
        pred_top_k = pred_sort[:, :, :topk]  # 上位Xのindex取得

        judges = []
        for i in range(len(tokens) - 2):
            pred_top_k_word = self.tokenizer.convert_ids_to_tokens(pred_top_k[i][i + 1])
            judges.append(tokens[i + 1] in pred_top_k_word)
            # logger.info(f'{tokens[i + 1]}: {judges[-1]}')
            # logger.debug(f'top k word={pred_top_k_word}')

        return all(judges)

    def check_threshold(self, sentence: str, threshold: float = 0.01):
        """
        [MASK]に対して予測された単語のスコアが閾値以上の単語群に、元の単語が含まれていればTrueと判定
        """
        tokens, predictions = self.mask_prediction(sentence)

        predictions = predictions.softmax(dim=2)

        judges = []
        for i in range(len(tokens) - 2):
            indices = (predictions[i][i + 1] >= threshold).nonzero()
            pred_top_word = self.tokenizer.convert_ids_to_tokens(indices)
            judges.append(tokens[i + 1] in pred_top_word)
            logger.info(f'{tokens[i + 1]}: {judges[-1]}')

        return all(judges)

In [14]:
import logging

logging.basicConfig(level=logging.INFO)
saved_path = "../../corpus/"
# saved_path = ""
PRETRAINED_MODEL = saved_path+'cl-tohoku/bert-base-japanese-whole-word-masking'
proofreader = BertProofreader(PRETRAINED_MODEL)

Some weights of the model checkpoint at ../../corpus/cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [15]:
proofreader.check_topk('絵がとても上手い', topk=100)

True

In [16]:
# diP5RUWTKaa8mYv

In [17]:
y_pred = []
for rdf in rdf_list:
    if rdf=="":
        y_pred.append(0)
        continue
    if proofreader.check_topk(rdf, topk=100):
        y_pred.append(0)
    else:
        y_pred.append(1)

RuntimeError: CUDA out of memory. Tried to allocate 2.02 GiB (GPU 0; 6.00 GiB total capacity; 2.43 GiB already allocated; 0 bytes free; 3.68 GiB reserved in total by PyTorch)

In [ ]:
y_pred.count(0)

130

In [ ]:
len(y_pred)

476

In [19]:
for s, r in zip(sys_utt, rdf_list):
    print(s, r)

海に行かないんですね？ 海に行く
海は大丈夫ですね 海は大丈夫です
海は危険です 海は危険です
海はいけないです 海はいける
わあ！いいですね！ 
ご存知ですいます 
ご存知になられます 御存知になる
スイカは実がなってたことあります 実がなる。スイカはある
あなたの好きな映画について教えてください あなたの好きだ。映画に教える
映画にはまるのです 映画にはまる
映画が楽しみです 
映画が楽しみですね 
映画館はチケットを買うのです 映画館はチケットを買う
食事で摂るのです 食事で摂る
外でとります 外でとる
外食は二人で行きますねぇ 外食は二人で行く
食事のパートナーはいてるんですね。外食で焼肉に行きますねぇ 食事パートナーはいる。外食で焼き肉に行く
あなた、外食のときも私を連れて行ってよ。 私を連れる。外食時も行く
夕食は明日の活力を生むから、しっかり食べてね。 夕食は明日活力を生む
野菜はカロリーが低いですよねえ 野菜はカロリーが低いです
カロリーは欲しいですね カロリーは欲しいです
ラーメンが食べたいです？ ラーメンが食べる
ラーメンは美味しいですね ラーメンは美味しいです
麺が美味しいですね 麺が美味しいです
パスタが好きですね パスタが好きです
パスタはクリーム系が好きですね パスタはクリーム系が好きです
海は楽しいですね 海は楽しいです
海はサイコーですね 海は最高です
海はキレーですね 海はキレーです
魚はおいんですか？？海は素晴らしいですね 魚はおく。海は素晴らしいです
海は素晴らしいですね 海は素晴らしいです
台風が凄いです 台風が凄いです
熱中症はいいですね 熱中症はいいです
熱中症は大丈夫ですね 熱中症は大丈夫です
手洗いうがいで予防が大切ですよねえ 手洗い嗽で予防が大切です
予防は手洗いが基本なんですよねー 
予防は難しいですね 予防は難しいです
海に行かないんですね？ 海に行く
海に行ってるんですね。海は一人で行ってます？ 海に行く。海は一人で行く
家族で買い物に行きます？ 家族で買い物に行く
家族でどこかに出かけます？ 家族でどこかどこに出かける
家族で食べに行きますか？ 家族で行く
家族で初詣に行ってるんですね。家族で旅行に行ってるのですか？ 家族で初詣でに行く。家族で旅行に行く
家族は夕食を食べてるのですか？ 家族は夕食を食べる
ケ